In [1]:
## import essential modules 
import qumcmc 
from qumcmc.basic_utils import *
from qumcmc.energy_models import IsingEnergyFunction

from qumcmc.classical_mcmc_routines import *
from qumcmc.quantum_mcmc_routines_qulacs import *     #for Qulacs Simulator backend
# from qumcmc.quantum_mcmc_routines_qulacs import quantum_enhanced_mcmc   #for qiskit Aer's Simulator backend 

from qumcmc.trajectory_processing import *

In [2]:
to_binary = lambda state_obtained, n_spins : f"{state_obtained:0{n_spins}b}"

In [3]:
# define the model
np.random.seed(610358)# should always be in the same cell!  
n_spins = 10

## construct problem Hamiltonian ##
shape_of_J=(n_spins,n_spins)

## defining J matrix (mutual 1-1 interaction)
# J =  np.round(np.random.choice([+1, 0, -1], size=(n_spins, n_spins)), decimals=2) 
J =  np.random.uniform(low= -2, high= 2, size= shape_of_J )

J = 0.5 * (J + J.transpose() )
J = np.round( J - np.diag(np.diag(J)) , decimals= 3)

# defining h
h = np.round(0.5 * np.random.randn(n_spins), decimals=2)
#h = np.round(np.random.uniform(low= -1, high = 1, size= (n_spins)), decimals=2)

# instantiate the model
model = IsingEnergyFunction(J, h, name= 'my_model')


#### Qulacs Exps

In [5]:
from qulacs import QuantumState

n = 2
state = QuantumState(n)
state.set_Haar_random_state(0)

# Get quantum bit numbers
qubit_count = state.get_qubit_count()
print("qubit_count", qubit_count)

# Get the probability that the specified qubit will be measured as 0
prob = state.get_zero_probability(1)
print("zero_prob_1", prob)


qubit_count 2
zero_prob_1 0.14831580678487433


In [6]:
np.absolute(state.get_amplitude(0)) , np.absolute( state.get_vector())

(0.2637259526321412, array([0.26372595, 0.28065001, 0.44726233, 0.80724259]))

In [7]:
1 - state.get_zero_probability(1)

0.8516841932151257

In [8]:

# Get arbitrary marginal probabilities
# Argument is an array of the same length as the number of qubits
# Specify 0,1,2. 0,1 is the probability of the subscript measured at that value
# 2 means that bit is peripheralized.
# For example, calculation of the probability that the third is measured as 0 and the 0th is measured as 1:
prob = state.get_marginal_probability([2,1])
print("marginal_prob", prob)


marginal_prob 0.8516841932151257


In [9]:

# Get the entropy of the probability distribution when measured on the Z basis
ent = state.get_entropy()
print("entropy", ent)

# Get squared norm (<a|a>)
# Because the operation maybe not Trace preserving, the norm of state does not necessarily to be 1.
sq_norm = state.get_squared_norm()
print("sqaured_norm", sq_norm)

# The number of measurements and sampling of all qubits Z-basis is given by the argument.
# Get a list of integers converted from the resulting binaries.
samples = state.sampling(10)
print("sampling", samples)

# Get a character string indicating whether the state vector is on CPU or GPU
dev_type = state.get_device_name()
print("device", dev_type)

entropy 0.9865530516551664
sqaured_norm 1.0
sampling [3, 2, 2, 3, 2, 3, 2, 3, 2, 3]
device cpu


In [10]:
[to_binary(s,2) for s in samples]

['11', '10', '10', '11', '10', '11', '10', '11', '10', '11']

In [11]:
state.sampling(2)

[3, 2]

#### Prepare new data objects

In [110]:
@dataclass
class MCMCState:
    
    var: str
    fixed: str
    
    accepted: bool = False
    
    def __post_init__(self):
        self.bitstring = self.var + self.fixed
        self.len_var = len(self.var)
        self.len_fixed = len(self.fixed)
        self.len = len(self.bitstring)
    
    def _update_var(self, new_state:str):
        if len(new_state) == self.len_var:
            self.var = new_state
            self.bitstring = self.var + self.fixed
        else : raise ValueError("Updated 'var' should be of len "+str(self.len_var))
    

In [111]:
t = MCMCState("100", "11"); t

MCMCState(var='100', fixed='11', accepted=False)

In [45]:
t._update_var('111')

#### Modify `run_qc_quantum_step`

In [49]:
def run_qc_quantum_step(
    qc_initialised_to_s: QuantumCircuit, model: IsingEnergyFunction, alpha, n_spins: int
) -> str:

    """
    Takes in a qc initialized to some state "s". After performing unitary evolution U=exp(-iHt)
    , circuit is measured once. Function returns the bitstring s', the measured state .

    ARGS:
    ----
    qc_initialised_to_s:
    model:
    alpha:
    n_spins:
    
    """

    h = model.get_h
    J = model.get_J

    # init_qc=initialise_qc(n_spins=n_spins, bitstring='1'*n_spins)
    gamma = np.round(np.random.uniform(0.25, 0.6), decimals=2)
    time = np.random.choice(list(range(2, 12)))  # earlier I had [2,20]
    delta_time = 0.8 
    num_trotter_steps = int(np.floor((time / delta_time)))
    qc_evol_h1 = fn_qc_h1(n_spins, gamma, alpha, h, delta_time)
    qc_evol_h2 = fn_qc_h2(J, alpha, gamma, delta_time=delta_time)
    trotter_ckt = trottered_qc_for_transition(
        n_spins, qc_evol_h1, qc_evol_h2, num_trotter_steps=num_trotter_steps
    )
    qc_for_mcmc = combine_2_qc(qc_initialised_to_s, trotter_ckt)# i can get rid of this!
    # run the circuit
    q_state=QuantumState(qubit_count=n_spins)
    q_state.set_zero_state()
    qc_for_mcmc.update_quantum_state(q_state)

    state_obtained=q_state.sampling(sampling_count=100)
    state_obtained=[ f"{state:0{n_spins}b}" for state in state_obtained]

    ## add postselection ##
    


    return state_obtained

In [21]:
d = MCMCState('10101', '110' , True)

In [24]:
def quantum_enhanced_mcmc(
    n_hops: int,
    model: IsingEnergyFunction,
    # alpha,
    initial_state: Optional[str ] = None,
    temperature=1,
):
    """
    version 0.2
    
    ARGS:
    ----
    Nhops: Number of time you want to run mcmc
    model:
    return_last_n_states:
    return_both:
    temp:

    RETURNS:
    -------
    Last 'return_last_n_states' elements of states so collected (default value=500). one can then deduce the distribution from it!
    
    """
    num_spins = model.num_spins

    if initial_state is None:
        initial_state = MCMCState(get_random_state(num_spins), accepted=True)
    else:
        initial_state = MCMCState(initial_state, accepted=True)
    
    current_state: MCMCState = initial_state
    energy_s = model.get_energy(current_state.bitstring)
    print("starting with: ", current_state.bitstring, "with energy:", energy_s)

    mcmc_chain = MCMCChain([current_state])

    print(mcmc_chain)
    for _ in tqdm(range(0, n_hops), desc='runnning quantum MCMC steps . ..' ):
        # get sprime
        qc_s = initialise_qc(n_spins= model.num_spins, bitstring=current_state.bitstring)
        
        s_prime = run_qc_quantum_step(
            qc_initialised_to_s=qc_s, model=model, alpha=model.alpha, n_spins= model.num_spins
        )
        # accept/reject s_prime
        energy_sprime = model.get_energy(s_prime)
        accepted = test_accept(
            energy_s, energy_sprime, temperature=temperature
        )
        mcmc_chain.add_state(MCMCState(s_prime, accepted))
        if accepted:
            current_state = mcmc_chain.current_state
            energy_s = model.get_energy(current_state.bitstring)

    return mcmc_chain 

In [129]:
from typing import Union

@dataclass
class RestrictedSampling:
        
        model : IsingEnergyFunction
        iterations : int = 10000
        temperature : float = 1.00
        initial_state : Optional[Union[ str, MCMCState]] = None
        
                
        def __post_init__(self):
                
                if self.initial_state is None : 
                        self.initial_state = MCMCState(get_random_state(model.num_spins), accepted=True)
                elif not isinstance(self.initial_state, MCMCState):
                        self.initial_state = MCMCState(self.initial_state, accepted=True)
                
                self.current_state: MCMCState = self.initial_state
                
                self.mcmc_chain: MCMCChain = MCMCChain([self.current_state])

        def run_classical_mcmc(self):
                
                energy_s = model.get_energy(self.current_state.bitstring)
                print('current state: ', self.current_state)
                for _ in tqdm(range(0, self.iterations), desc= 'running MCMC steps ...'):
                        # get sprime #
                        s_prime = MCMCState(get_random_state(self.current_state.len_var  ), self.current_state.fixed)
                        print('s_prime:', s_prime)
                        
                        # accept/reject s_prime
                        energy_sprime = model.get_energy(s_prime.bitstring)   # to make this scalable, I think you need to calculate energy ratios.
                        accepted = test_accept(
                        energy_s, energy_sprime, temperature=self.temperature
                        )
                        if accepted:
                                s_prime.accepted = accepted
                                self.current_state = s_prime
                                print('current state: ', self.current_state)
                                energy_s = model.get_energy(self.current_state.bitstring)
                        
                        self.mcmc_chain.add_state(s_prime)

                return self.mcmc_chain



In [130]:
state = MCMCState('101001','1110', accepted= True)

In [131]:
rs1 = RestrictedSampling(model, initial_state= state)

In [137]:
smp.get_accepted_dict(normalize=True)

Counter({'1010011110': 0.0024752475247524753,
         '1110101110': 0.009900990099009901,
         '0010111110': 0.24752475247524752,
         '1010101110': 0.09900990099009901,
         '1010111110': 0.14603960396039603,
         '0110111110': 0.43316831683168316,
         '1110111110': 0.03217821782178218,
         '0010011110': 0.009900990099009901,
         '1100101110': 0.017326732673267328,
         '1000101110': 0.0024752475247524753})

In [132]:
smp = rs1.run_classical_mcmc()

current state:  MCMCState(var='101001', fixed='1110', accepted=True)


running MCMC steps ...:   9%|▉         | 941/10000 [00:00<00:00, 9406.11it/s]

s_prime: MCMCState(var='000001', fixed='1110', accepted=False)
s_prime: MCMCState(var='000000', fixed='1110', accepted=False)
s_prime: MCMCState(var='111000', fixed='1110', accepted=False)
s_prime: MCMCState(var='100011', fixed='1110', accepted=False)
s_prime: MCMCState(var='011000', fixed='1110', accepted=False)
s_prime: MCMCState(var='111100', fixed='1110', accepted=False)
s_prime: MCMCState(var='010110', fixed='1110', accepted=False)
s_prime: MCMCState(var='111010', fixed='1110', accepted=False)
current state:  MCMCState(var='111010', fixed='1110', accepted=True)
s_prime: MCMCState(var='000011', fixed='1110', accepted=False)
s_prime: MCMCState(var='110011', fixed='1110', accepted=False)
s_prime: MCMCState(var='001011', fixed='1110', accepted=False)
current state:  MCMCState(var='001011', fixed='1110', accepted=True)
s_prime: MCMCState(var='101110', fixed='1110', accepted=False)
s_prime: MCMCState(var='110100', fixed='1110', accepted=False)
s_prime: MCMCState(var='100110', fixed='111

running MCMC steps ...:  22%|██▏       | 2204/10000 [00:00<00:00, 11289.79it/s]

s_prime: MCMCState(var='110111', fixed='1110', accepted=False)
s_prime: MCMCState(var='100010', fixed='1110', accepted=False)
s_prime: MCMCState(var='101011', fixed='1110', accepted=False)
current state:  MCMCState(var='101011', fixed='1110', accepted=True)
s_prime: MCMCState(var='111011', fixed='1110', accepted=False)
s_prime: MCMCState(var='111110', fixed='1110', accepted=False)
s_prime: MCMCState(var='100111', fixed='1110', accepted=False)
s_prime: MCMCState(var='001010', fixed='1110', accepted=False)
s_prime: MCMCState(var='101110', fixed='1110', accepted=False)
s_prime: MCMCState(var='111011', fixed='1110', accepted=False)
s_prime: MCMCState(var='010000', fixed='1110', accepted=False)
s_prime: MCMCState(var='101111', fixed='1110', accepted=False)
s_prime: MCMCState(var='111111', fixed='1110', accepted=False)
s_prime: MCMCState(var='010011', fixed='1110', accepted=False)
s_prime: MCMCState(var='000110', fixed='1110', accepted=False)
s_prime: MCMCState(var='000101', fixed='1110', ac

...

s_prime: MCMCState(var='100011', fixed='1110', accepted=False)
s_prime: MCMCState(var='010111', fixed='1110', accepted=False)
s_prime: MCMCState(var='110101', fixed='1110', accepted=False)
s_prime: MCMCState(var='110111', fixed='1110', accepted=False)
s_prime: MCMCState(var='110101', fixed='1110', accepted=False)
s_prime: MCMCState(var='111110', fixed='1110', accepted=False)
s_prime: MCMCState(var='001010', fixed='1110', accepted=False)
s_prime: MCMCState(var='001111', fixed='1110', accepted=False)
s_prime: MCMCState(var='011101', fixed='1110', accepted=False)
s_prime: MCMCState(var='100111', fixed='1110', accepted=False)
s_prime: MCMCState(var='001010', fixed='1110', accepted=False)
s_prime: MCMCState(var='110000', fixed='1110', accepted=False)
s_prime: MCMCState(var='101011', fixed='1110', accepted=False)
current state:  MCMCState(var='101011', fixed='1110', accepted=True)
s_prime: MCMCState(var='111010', fixed='1110', accepted=False)
s_prime: MCMCState(var='100001', fixed='1110', ac

running MCMC steps ...:  72%|███████▏  | 7220/10000 [00:00<00:00, 12344.75it/s]

s_prime: MCMCState(var='001100', fixed='1110', accepted=False)
s_prime: MCMCState(var='110000', fixed='1110', accepted=False)
s_prime: MCMCState(var='100000', fixed='1110', accepted=False)
s_prime: MCMCState(var='000111', fixed='1110', accepted=False)
s_prime: MCMCState(var='000100', fixed='1110', accepted=False)
s_prime: MCMCState(var='111111', fixed='1110', accepted=False)
s_prime: MCMCState(var='001001', fixed='1110', accepted=False)
s_prime: MCMCState(var='011100', fixed='1110', accepted=False)
s_prime: MCMCState(var='011000', fixed='1110', accepted=False)
s_prime: MCMCState(var='000111', fixed='1110', accepted=False)
s_prime: MCMCState(var='011101', fixed='1110', accepted=False)
s_prime: MCMCState(var='100001', fixed='1110', accepted=False)
s_prime: MCMCState(var='100000', fixed='1110', accepted=False)
s_prime: MCMCState(var='100100', fixed='1110', accepted=False)
s_prime: MCMCState(var='011110', fixed='1110', accepted=False)
s_prime: MCMCState(var='011110', fixed='1110', accepted

running MCMC steps ...: 100%|██████████| 10000/10000 [00:00<00:00, 12218.33it/s]

s_prime: MCMCState(var='101001', fixed='1110', accepted=False)
s_prime: MCMCState(var='001101', fixed='1110', accepted=False)
s_prime: MCMCState(var='010100', fixed='1110', accepted=False)
s_prime: MCMCState(var='001101', fixed='1110', accepted=False)
s_prime: MCMCState(var='000001', fixed='1110', accepted=False)
s_prime: MCMCState(var='101011', fixed='1110', accepted=False)
s_prime: MCMCState(var='000011', fixed='1110', accepted=False)
s_prime: MCMCState(var='010110', fixed='1110', accepted=False)
s_prime: MCMCState(var='110001', fixed='1110', accepted=False)
s_prime: MCMCState(var='100111', fixed='1110', accepted=False)
s_prime: MCMCState(var='100000', fixed='1110', accepted=False)
s_prime: MCMCState(var='100101', fixed='1110', accepted=False)
s_prime: MCMCState(var='010100', fixed='1110', accepted=False)
s_prime: MCMCState(var='001100', fixed='1110', accepted=False)
s_prime: MCMCState(var='011101', fixed='1110', accepted=False)
s_prime: MCMCState(var='011101', fixed='1110', accepted